In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text_corpus = """
Say my name.” “Tread lightly.” “I am the one who knocks.”
Evelyn Thorne (50s), a world-famous violinist known for her fiery passion and technical brilliance, is preparing for the biggest concert of her career – a solo performance at Carnegie Hall. However, beneath the surface, Evelyn carries the weight of a past shrouded in tragedy.
Years ago, a devastating fire claimed the life of her younger sister, Lily, who also possessed a remarkable musical talent. Evelyn, the lone survivor, channeled her grief into her music, achieving stardom but at the cost of emotional isolation.
Enter Sophia (18), Evelyn's estranged daughter. A gifted violinist in her own right, Sophia resents her mother's relentless push for perfection and the constant comparisons to the ghost of her aunt, Lily. Their dysfunctional relationship threatens to derail both their musical aspirations.
As the pressure mounts for the Carnegie Hall performance, a series of events forces Evelyn and Sophia to confront their painful past. A mysterious fire destroys Evelyn's prized violin, a gift from Lily. A surprise visit from a former mentor reveals a long-held secret about the night of the fire. Sophia receives an invitation to an esteemed music school, an opportunity that could finally separate her from her mother's shadow.
Evelyn, facing the limitations of her own mortality and the destructive impact of her choices, embarks on a journey of reconciliation. She seeks to connect with Sophia, not just as a musician, but also as a mother seeking forgiveness. They delve into their shared history, uncovering new truths about the fire and the life of Lily.
The climax arrives with Carnegie Hall looming. Sophia grapples with the weight of expectation, her own desires, and the complex emotions she harbors for her mother. Evelyn, vulnerable and raw, must confront her deepest fears and the ghost of her past.
Will they be able to mend their broken bond and find solace in their shared passion for music? Will Sophia choose her own path or succumb to the pressure of living up to her mother and aunt's legacy? The climactic performance at Carnegie Hall becomes a poignant culmination of their journey, a testament to the power of music to heal and the enduring love between a mother and daughter.
"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_corpus])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text_corpus.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [8]:
learning_rate = 0.01
batch_size = 50
epochs = 10

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [10]:
model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/10
8/8 [==============================] - 10s 231ms/step - loss: 5.2322 - accuracy: 0.0515
Epoch 2/10
8/8 [==============================] - 2s 215ms/step - loss: 4.9699 - accuracy: 0.0488
Epoch 3/10
8/8 [==============================] - 2s 240ms/step - loss: 4.7563 - accuracy: 0.0678
Epoch 4/10
8/8 [==============================] - 2s 263ms/step - loss: 4.4667 - accuracy: 0.0840
Epoch 5/10
8/8 [==============================] - 2s 311ms/step - loss: 3.9470 - accuracy: 0.1978
Epoch 6/10
8/8 [==============================] - 2s 200ms/step - loss: 3.2405 - accuracy: 0.2846
Epoch 7/10
8/8 [==============================] - 1s 161ms/step - loss: 2.5049 - accuracy: 0.4065
Epoch 8/10
8/8 [==============================] - 1s 150ms/step - loss: 1.7891 - accuracy: 0.5935
Epoch 9/10
8/8 [==============================] - 1s 144ms/step - loss: 1.1829 - accuracy: 0.7507
Epoch 10/10
8/8 [==============================] - 1s 149ms/step - loss: 0.7491 - accuracy: 0.8699


In [11]:
import numpy as np

def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list)[0]
        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word
    return seed_text

In [12]:
generated_text = generate_text("Generated Text:", 10, model, max_sequence_length)
print(generated_text)

1/1 [==============================] - 0s 34ms/step
Generated Text: thorne 50s a musician but a violinist known for her


In [13]:
val_loss, val_accuracy = model.evaluate(X, y, verbose=0)
print(f"Loss: {val_loss:.4f}")
print(f" Accuracy: {val_accuracy:.4f}")

Loss: 0.4613
 Accuracy: 0.9512
